In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv', sep='\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df['message']

In [8]:
y = df['label']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
count_vect = CountVectorizer()

In [12]:
# Fit vectorizer to the data (build a vocab, count the number of words...)
# count_vect.fit(X_train)
# Transform the original text message --> Vector
# X_train_counts = count_vect.transform(X_train)

X_train_counts = count_vect.fit_transform(X_train)

In [13]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [14]:
X_train_counts.shape

(3733, 7082)

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer

In [16]:
tfidf_transformer = TfidfTransformer()

In [17]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [18]:
X_train_tfidf.shape

(3733, 7082)

In [19]:
# The two steps above CountVectorizer & TfidfTransformer
# were combined into one feature TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
vectorizer = TfidfVectorizer()

In [21]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [22]:
X_train_tfidf.shape

(3733, 7082)

In [23]:
from sklearn.svm import LinearSVC

In [24]:
clf = LinearSVC()

In [25]:
clf.fit(X_train_tfidf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [26]:
# We can combine all the steps above in one single pipeline step
# so it is easier to call predict without the need to vectorize and
# transform the test data set
from sklearn.pipeline import Pipeline

In [27]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [28]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [29]:
predictions = text_clf.predict(X_test)

In [30]:
from sklearn.metrics import confusion_matrix, classification_report

In [31]:
print(confusion_matrix(y_test, predictions))

[[1586    7]
 [  12  234]]


In [32]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [33]:
# Testing with random text
text_clf.predict(['Hi, how are you doing today?'])

array(['ham'], dtype=object)

In [34]:
text_clf.predict(['Congratulations! You have ben selected as a winner. Text WON to 44255.'])

array(['spam'], dtype=object)